# Analyzing UNICEF Global Education Data

In this notebook, I am using data provided by UNICEF to describe and map some key education metrics. I will provide supplementary analysis (maps, most likely) in R in the directory. I am particularly interested in visualizing these metrics to see global education disparities and perhaps see if any other indicators correlate with these education metrics. The raw dataset is located here, but I did some cleaning in Excel to make the parsing into Pandas easier: http://data.unicef.org/resources/state-worlds-children-2016-statistical-tables/

In [30]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import axes
import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
from IPython.display import SVG, display

In [2]:
dataframe = pd.read_csv('SOWC-2016_all-tables_261.csv', na_values=['-'], skip_blank_lines=False, encoding = "ISO-8859-1")

In [3]:
dataframe

,Country or Area,Male Youth Literacy Rate,Female Youth Literacy Rate,Mobile Phones (per 100),Internet Users (per 100),Male gross enrolment ratio pre-primary school,Female gross enrolment ratio pre-primary school,Male gross enrolment ratio primary school,Female gross enrolment ratio primary school,Male net enrolment ratio primary school,...,Male net attendance ratio primary school,Female net attendance ratio primary school,Out-of-school children of primary school age rate,Out-of-school children of primary school age number (000),"Survival Rate to last primary grade, admin","Survival rate to last primary grade, survey",Male net enrolment ratio secondary school,Female net enrolment ratio secondary school,Male net attendance ratio secondary school,Female net attendance ratio secondary school
0,Afghanistan,62,32,75,6,,,131,92,,...,62,46,,,,84,62,35,47,27
1,Albania,99,99,105,60,90,87,114,111,,...,92,93,4,7,99,100,86,85,78,79
2,Algeria,94,89,93,18,79,79,122,115,,...,98,97,1,36,93,98,,,77,81
3,Andorra,,,83,96,,,,,,...,,,,,71,,,,,
4,Angola,79,67,63,21,64,94,157,100,95,...,77,75,16,624,32,83,14,11,21,17
5,Antigua and Barbuda,,,132,64,78,72,101,93,87,...,,,14,1,,,78,81,,
6,Argentina,99,99,159,65,71,72,111,110,,...,98,99,0,6,95,97,,,82,85
7,Armenia,100,100,116,46,46,60,96,111,,...,100,99,,,90,100,82,97,92,93
8,Australia,,,131,85,110,108,107,106,97,...,,,3,51,,,86,88,,
9,Austria,,,152,81,102,101,103,102,,...,,,,,99,,,,,


Great, now we have our dataset loaded into pandas. We see there are a wide variety of indicators that we can look at from a global perspective. At the highest level, let's first calculate some basic statistics for each attribute. The data is stored as strings so I will have to cast them to numeric to run statistics. Note that there is missing data in this dataset, so the values may not be entirely accurate.

In [4]:
#make list of numeric columns for statistics
numeric_columns = []
for column in dataframe:
    if(column!='Country or Area'):
        numeric_columns.append(column)
print("{: <60} {: >10} {: >10}".format('Attribute', 'Mean', 'Median'))

for col in numeric_columns:
    mean = np.mean(pd.to_numeric(dataframe[col], errors='coerce'))
    median = np.nanmedian(pd.to_numeric(dataframe[col], errors='coerce'))
    print("{: <60} {: >10} {: >10}".format(col, str(round(mean,3)), str(median)))

Attribute                                                          Mean     Median
Male Youth Literacy Rate                                         89.846       97.0
Female Youth Literacy Rate                                       86.631       98.0
Mobile Phones (per 100)                                         103.531      106.5
Internet Users (per 100)                                         43.691       43.0
Male gross enrolment ratio pre-primary school                    62.793       70.5
Female gross enrolment ratio pre-primary school                   63.23       70.5
Male gross enrolment ratio primary school                       106.217      105.0
Female gross enrolment ratio primary school                     103.457      103.0
Male net enrolment ratio primary school                          90.764       95.0
Female net enrolment ratio primary school                        89.764       95.0
Male net attendance ratio primary school                         85.453       92.0
Fema

On their own, these global statistics do not mean much, but some interesting things do stand out. For example, the youth literacy rates are quite high! Additionally, I found it interesting that while globally there are about 103 mobile phones per 100 people, there are only around 43 internet users per 100 people. This shows an interesting discrepancy between the prevalence of mobile phones vs. a lack of internet access. Let's see which countries have the minimum and maximum value for each indicator to see if any trends emerge. Note that many countries or areas may have the same rates, but here only one is being selected.

In [5]:
print("{: <60} {: >20} {: >30}".format('Attribute', 'Max Country/Area', 'Min Country/Area'))
for col in numeric_columns:
    minCountry = dataframe.loc[pd.to_numeric(dataframe[col], errors='coerce').idxmin()]['Country or Area']
    maxCountry = dataframe.loc[pd.to_numeric(dataframe[col], errors='coerce').idxmax()]['Country or Area']
    #truncate names for printing 
    if(len(minCountry) > 20):
        minCountry = minCountry[0:20]
    
    if(len(maxCountry) > 20):
        maxCountry = maxCountry[0:20]
    print("{: <60} {: >20} {: >30}".format(col, maxCountry, minCountry))


Attribute                                                        Max Country/Area               Min Country/Area
Male Youth Literacy Rate                                                  Armenia                          Niger
Female Youth Literacy Rate                                                Armenia                          Niger
Mobile Phones (per 100)                                                    Kuwait                        Eritrea
Internet Users (per 100)                                                  Iceland           Democratic People's 
Male gross enrolment ratio pre-primary school                             Ecuador                           Chad
Female gross enrolment ratio pre-primary school                           Ecuador                           Chad
Male gross enrolment ratio primary school                                   Haiti                        Eritrea
Female gross enrolment ratio primary school                                 Haiti               

Let's plot the worst-performing countries in red and the best-performing countries in green to see the geospatial distribution. The R code used to develop these maps is in the directory as performance.Rmd. (Sidenote: I tried doing these in Python natively but R support for maps is just so much better!)

In [ ]:
#aggregate the best and worst-performing countries: the names were changed to ISO3 format for easy joining 
best_performing = ['ARM', 'KWT', 'ISL', 'ECU', 'HTI', 'CMR', 'HRV', 'NIU', 'JPN'
                   , 'ALB', 'QAT', 'BRB', 'PRK', 'BLR', 'ARG'] 
worst_performing = ['NER', 'ERI', 'PRK', 'TCD', 'LBR', 'BFA', 'PLW', 'UGA'
                   , 'COM', 'AGO', 'CAF', 'SOM']

#Construct dataframes from the lists so that we can write it to csv and pass to R for mapping - insert dummy valu
df1 = pd.DataFrame({'Country': best_performing, 'Value' : np.ones(len(best_performing))})
df2 = pd.DataFrame({'Country': worst_performing, 'Value' : np.ones(len(worst_performing))})
df1.to_csv('best_performing.csv')
df2.to_csv('worst_performing.csv')

As we can see in the attached maps, the worst-performing countries are all located in Africa, with the exception of North Korea. The best-performing countries are not bound by region. It may be enlightening to focus analysis on Africa to learn what factors can be improved and to see what additional data can be brought in to learn more about their effects on education outcomes. 